In [1]:
import gensim
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import numpy
from random import shuffle
import nltk

In [2]:
queryTrain = []
passageTrain = []

def label_file(fr, qIdx, pIdx):
    fw_q = open("D:/demo/queryLabel.tsv", 'w', encoding='utf-8')
    fw_p = open("D:/demo/passageLabel.tsv", 'w', encoding='utf-8')
    
    with open(fr, 'r', encoding='utf-8') as fr:
        for item_no, line in enumerate(fr):
            arr = line.split('\t')
            query = arr[qIdx]
            passage = arr[pIdx]
            queryTrain.append(LabeledSentence(utils.to_unicode(query).split(), ["query" + "_%s"%item_no]))
            fw_q.write("{0}\t{1}".format("query" + "_%s"%item_no, line))
            
            passageTrain.append(LabeledSentence(utils.to_unicode(passage).split(), ["passage" + "_%s"%item_no]))
            fw_p.write("{0}\t{1}".format("passage_%s"%item_no, line))
    
    fw_q.close()
    fw_p.close()

In [3]:
class LabelLineSentence(object):
    def __init__(self, source):
        self._source = source
        
        #make sure the labels are unique
        flipped = {}
        for fn, label in self._source.items():
            if label in flipped:
                raise Exception("Non-unique prefix encountered!")
            else:
                flipped[label] = fn
            
        
    def __iter__(self):
        for fn, prefix in self._source.items():
            with open(fn, 'r', encoding='utf-8') as fr:
                for item_no, line in enumerate(fr):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + "_%s"%item_no])
                    
    def to_array(self):
        self.sentences = []
        for fn, prefix in self._source.items():
            with open(fn, 'r', encoding='utf-8') as fr:
                for item_no, line in enumerate(fr):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + "_%s"%item_no]))    
        return self.sentences
    
    def setences_perm(self):
        shuffle(self.sentences)
        return self.sentences
        
     

In [4]:
label_file("D:/demo/JudgeResult.tsv", 0, 2)

In [5]:
queryTrain[:10]

[LabeledSentence(words=['activate', 'imessage', 'iphone', '6', 'plus'], tags=['query_0']),
 LabeledSentence(words=['activate', 'imessage', 'iphone', '6', 'plus'], tags=['query_1']),
 LabeledSentence(words=['water', 'conflicts'], tags=['query_2']),
 LabeledSentence(words=['installing', 'windows', '10', 'vmware'], tags=['query_3']),
 LabeledSentence(words=['installing', 'windows', '10', 'vmware'], tags=['query_4']),
 LabeledSentence(words=['season', 'corn', 'beef'], tags=['query_5']),
 LabeledSentence(words=['morosely', 'adj', 'definition'], tags=['query_6']),
 LabeledSentence(words=['morosely', 'adj', 'definition'], tags=['query_7']),
 LabeledSentence(words=['average', 'salary', 'without', 'college', 'degree'], tags=['query_8']),
 LabeledSentence(words=['federal', 'fiduciary', 'definition'], tags=['query_9'])]

In [6]:
passageTrain[:10]

[LabeledSentence(words=['following', 'guides', 'activate', 'iMessage', 'iPhone', '6,', 'iPhone', '6', 'Plus,', 'iPhone', '5s,', 'iPhone', '5c,', 'iPhone', '5,', 'iPhone', '4s,', 'iPhone', '4,', 'iPad', 'Air', '2,', 'iPad', 'Air,', 'iPad', 'Mini', '3,', 'iPad', 'Mini', '2,', 'iPad', 'Mini,', 'iPad', '4,', 'iPad', '3', 'iPad', '2.'], tags=['passage_0']),
 LabeledSentence(words=['following', 'guides', 'activate', 'iMessage', 'iPhone', '6,', 'iPhone', '6', 'Plus,', 'iPhone', '5s,', 'iPhone', '5c,', 'iPhone', '5,', 'iPhone', '4s,', 'iPhone', '4,', 'iPad', 'Air', '2,', 'iPad', 'Air,', 'iPad', 'Mini', '3,', 'iPad', 'Mini', '2,', 'iPad', 'Mini,', 'iPad', '4,', 'iPad', '3', 'iPad', '2.'], tags=['passage_1']),
 LabeledSentence(words=['Water', 'Conflict.', 'ongoing', 'effort', 'understand', 'connections', 'water', 'resources,', 'water', 'systems,', 'international', 'security', 'conflict,', 'Pacific', 'Institute', 'initiated', 'project', 'late', '1980s', 'track', 'categorize', 'events', 'related',

In [7]:
all_train = queryTrain

In [8]:
all_train.extend(passageTrain)

In [9]:
len(all_train)

8776

In [10]:
len(passageTrain)

4388

# Train Doc2Vec

In [11]:
model = Doc2Vec(min_count=1, window=5, size=100, sample=1e-4, negative=5, workers=8)

In [12]:
model.build_vocab(all_train)

In [13]:
for epoch in range(10):
    shuffle(all_train)
    model.train(all_train)

In [35]:
for i in range(10):
    score = model.docvecs.similarity("query_%d"%i, "passage_%d"%i)
    print("{0}\t{1}".format(i, score))

0	0.15508664736241742
1	0.17607638849604604
2	0.3457598302000365
3	0.43879796223253364
4	0.4299172362641229
5	0.3295498331421936
6	0.620914786211397
7	0.6328540594012924
8	0.2882902772010174
9	0.3240142177289195


In [15]:
len(all_train)/2

4388.0

# Add Feature

In [20]:
scores = []
for i in range(int(len(all_train)/2)):
    querylabel = "query_%d"%i
    passagelabel = "passage_%d"%i
    score = model.docvecs.similarity(querylabel, passagelabel)
    scores.append(score)
    

In [36]:
fr = open("D:/demo/JudgeResult.tsv", 'r', encoding='utf-8')
fw = open("D:/Project/Malta/QueryPassage_SimScore.tsv", 'w+', encoding='utf-8')

for item_no, line in enumerate(fr):
    score = scores[item_no]
    fw.write("{0}\t{1}\n".format(line.strip(), score))
    
fr.close()
fw.close()

In [37]:
len(all_train)

8776

# Model Save

In [38]:
model.save("D:/Project/Malta/SemanticFeatureModuleV0_1")

# SimScore analysis between perfect and bad data set

In [39]:
pos_simScore = []
neg_simScore = []

import collections

In [40]:
Schema = collections.namedtuple("Schema", "query url passage title judge simScore")


In [41]:

with open("D:/Project/Malta/QueryPassage_SimScore.tsv", 'r', encoding='utf-8') as fr:
    for line in fr:
        arr = line.split('\t')
        if(arr[4] == "Bad"):
            neg_simScore.append(float(arr[5]))
        elif(arr[4] =="Perfect"):
            pos_simScore.append(float(arr[5]))
                
        
        

In [42]:
len(neg_simScore), len(pos_simScore)

(1436, 2950)

In [150]:
shuffle(pos_simScore)
pos_simScore = pos_simScore[:1436]
len(neg_simScore), len(pos_simScore)

(1436, 1436)

In [151]:

import math


In [152]:
import numpy as np
from pandas import Series, DataFrame

In [153]:

def plt_gen(arr, v_num):
    for f in arr:
        f = str(f)
        f = f[:f.find('.') + 2]
        f = float(f)
        if (f not in v_num):
            v_num[f] = 0
        v_num[f] += 1


In [154]:
neg_plt_dic = {}
pos_plt_dic = {}
plt_gen(neg_simScore, neg_plt_dic)
plt_gen(pos_simScore, pos_plt_dic)

In [155]:
def x_y(dic, x_arr, y_arr):
    for key, num in dic.items():
        x_arr.append(key)
        y_arr.append(num)

In [157]:
pos_x_arr = []
pos_y_arr = []
neg_x_arr = []
neg_y_arr = []

x_y(neg_plt_dic, neg_x_arr, neg_y_arr)
x_y(pos_plt_dic, pos_x_arr, pos_y_arr)

In [158]:
pos_x_arr,neg_x_arr

([0.3, 0.5, 0.6, 0.8, -0.0, 0.7, 0.2, -0.1, 0.1, 0.4],
 [0.1, 0.3, 0.6, 0.8, 0.5, 0.7, 0.2, -0.1, 0.4, 0.0])

In [159]:
pos_series = Series(pos_y_arr, index = pos_x_arr)

neg_series = Series(neg_y_arr, index = neg_x_arr)

In [160]:
pos_series = pos_series.sort_index()
neg_series = neg_series.sort_index()

In [161]:
neg_series

-0.1      2
 0.0     62
 0.1    153
 0.2    273
 0.3    343
 0.4    292
 0.5    207
 0.6     76
 0.7     24
 0.8      4
dtype: int64

In [162]:
import pandas as pd

In [163]:
pos_neg_pd = pd.concat([pos_series, neg_series], axis=1)

In [165]:
pos_neg_pd.plot()

In [166]:
plt.show()